In [1]:
import tensorflow as tf
import gym
import numpy as np
from tensorflow_probability import distributions as dists
import tensorflow.keras.layers as kl

from rl_agents.env_utils import rollouts_generator, get_adv_vtarg
from rl_agents.ppo.policy import Actor, Critic
from rl_agents.ppo.agent import PPO_Agent

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

tf.random.set_seed(0)

In [2]:
env = gym.make('Pendulum-v0')
is_continuous = isinstance(env.action_space, gym.spaces.Box)
obs_dim = env.observation_space.shape
act_dim = env.action_space.shape

In [3]:
actor = Actor(obs_dim, act_dim, is_continuous)
critic = Critic(obs_dim)
vero = PPO_Agent(actor, critic)

In [24]:
generator = rollouts_generator(vero, env, horizon=2048)
rollout = generator.__next__()

In [27]:
advantage, target_value = get_adv_vtarg(rollout, lam=0.95, gamma=0.99)

In [30]:
vero.run_epoch(rollout['ob'], rollout['log_probs'], target_value, advantage, 2)

12.057407462052371
1.4344754579645111
1.8086110412886089
1.5526932434684402
76.898466862543643
1.4575256098540161
1.8084743073685678
1.5213524460723129
1.4745352992680492
2.229411451559872
1.1326168213630738
1.9302413811947037
2.4961163332943324
1.2610773066478154
1.4834187211384728
1.2394797323270765
1.5038401713364946
2.1653899525103997
1.6029983169297768
2.179096787071201
1.2297472126836955
1.4774434362152498
3.6722365902306793
1.6448151667234483
1.4554169570510214
5.8630185026872983
2.0068280810674262
2.7338492207262703
1.7230504766513068
1.3390250588214894
2.0177525759065316
1.713844090678627
12.836897617362387
1.310364990128803
4.0190836866340529
1.7140694572686612
81.146455174115985
1.6302569108329139
1.5673726572352229
1.5455202134876658
1.4819275200561304
1.9751938865202368
1.0432735764380359
1.4374316202188779
2.2947897744270822
1.2578767433399429
1.5143486720713366
1.2813299640138585
1.4745315958567897
2.1755903976639019
1.3245929161031929
3.5586144559715773
1.24753732035991

In [11]:
obs = env.reset()
l = vero.act_stochastic(obs)
print(l)

(<tf.Tensor: id=174588, shape=(1,), dtype=float64, numpy=array([0.68133348])>, <tf.Tensor: id=174592, shape=(1,), dtype=float64, numpy=array([-0.12700349])>, <tf.Tensor: id=174596, shape=(1,), dtype=float64, numpy=array([-1.09003816])>)


In [13]:
ac = env.action_space.sample()
print(ac.dtype)

float32


In [18]:
print(np.float64)

<class 'numpy.float64'>
